In [38]:
# !pip install selenium
# !pip install bs4
# !pip install beautifulsoup4
# !pip install chromedriver-autoinstaller
# pip install mysql-connector-python
# !pip install openpyxl

In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from requests import get
from lxml import html
from selenium import webdriver
import time
import re
from selenium import webdriver
import chromedriver_autoinstaller
import mysql.connector
from mysql.connector import Error
import os

chromedriver_autoinstaller.install()  

def removeChar(s,x):
    return s.replace(x, "")

In [15]:
user=input("Enter the name of the Car You want to buy:")

st=""
page=1
total_count=0
for i in user.split():
        st+=i + "+"
st = st[:-1]

while True:
    url=f'https://www.pakwheels.com/used-cars/search/-/?page={page}&q={st}'
    print("Link:",url,"\n")
    
#?page=4&q=corolla
    driver = webdriver.Chrome()
    driver.get(url)



    html = driver.page_source
    parsed_html = BeautifulSoup(html, "html.parser")

    #driver.close()

    
    containers = parsed_html.find_all("div", {"class" : "col-md-9 grid-style"})
    


    count=0
    name=[]
    city=[]
    info=[]
    info1=[]
    info2=[]
    info3=[]
    info4=[]
    info5=[]

    updated_info=[]
    price=[]
    price2=[]

    certified=[]
    managed_by=[]
    driver.close()
    for i in range(0,(len(containers))):
        count+=1
        first_product = containers[i]
        car_name=first_product.find('h3').text
        car_name=removeChar(car_name," for Sale")
        name.append(car_name)
    #list-unstyled search-vehicle-info fs13
        city_name=first_product.find('ul', class_ = 'list-unstyled search-vehicle-info fs13').text
        city_name = re.sub(r'[^a-zA-Z]', '', city_name)
        city.append(city_name)
    #price-details generic-dark-grey
        info_car=first_product.find('ul', class_ = 'list-unstyled search-vehicle-info-2 fs13').text
        count_info=1
        info_car=info_car.split()
        info1.append(int(info_car[0]))#year
        info2.append(info_car[1]+info_car[2])#KMs
        info3.append(info_car[3])#fuel
        info4.append(info_car[4]+info_car[5])#Horsepower
        info5.append(info_car[6])#transmission
        info.append(info_car[0] + " " + info_car[1] + info_car[2] + " " + info_car[3] + " " + info_car[4]+ info_car[5] + " " +info_car[6])

        updated_information=first_product.find('div', class_ = 'pull-left dated').text
        updated_info.append(updated_information)


        prices=first_product.find('div', class_ = 'price-details generic-dark-grey').text
        prices=re.sub(r'\s+', ' ', prices.strip())
        price.append(prices)



        cert=first_product.find(class_ = 'pull-left mr5')
        if(cert):
            certified.append("Certified")
        else:
            certified.append("Not Certified")
        
        manage=first_product.find('span', class_ = 'sold-by-pw')
        if(manage):
            managed_by.append("PakWheels")
        else:
            managed_by.append("Customer")
    #sold-by-pw
    price_number=[]
    for price_num in price:
        check=price_num.find('PKR')
        if(check==-1):
            price_number.append(0)
        else:
            price_num = price_num.replace('PKR', '')

            if(len(price_num.split())==1):
                price_number.append(price_num)
            else:
                number, unit = price_num.split()
                number = float(number)
                
                if unit == 'lacs':
                    number *= 100000
                elif unit == 'crore':
                    number *= 10000000

                price_number.append(int(number))

    
    
    if(page==1):
        df = pd.DataFrame(list(zip(name, price,price_number,city,info1,info2,info3,info4,info5,certified,managed_by)), columns =['Name', 'PriceInText','Price','City','Year','Distance','Fuel','HorsePower','Transmission','Certified','ManagedBy'])
        
        df['Distance2'] = df['Distance'].str.replace(',', '').str.replace('km', '')

       
        df['Distance2'] = pd.to_numeric(df['Distance2'])
        
    else:
        
        df2 = pd.DataFrame(list(zip(name, price,price_number,city,info1,info2,info3,info4,info5,certified,managed_by)), columns =['Name', 'PriceInText','Price','City','Year','Distance','Fuel','HorsePower','Transmission','Certified','ManagedBy'])
        df2['Distance2'] = df2['Distance'].str.replace(',', '').str.replace('km', '')
        df2['Distance2'] = pd.to_numeric(df2['Distance2'])
        
        df = pd.concat([df, df2], ignore_index = True, axis = 0)
    
    page+=1
    total_count+=count
    find_next_page=parsed_html.find("ul", {"class" : "pagination search-pagi"})
    try:
         find_next_page=parsed_html.find("li", {"class" : "next_page"})
         if find_next_page==None:
            print('No more Pages left')
            #print(names)
            break
    except AttributeError:
         pass





"""
Data Cleaning
"""


if os.path.exists("Results.xlsx"):
    data2=pd.read_excel("Results.xlsx")
    data1 = pd.concat([df, data2], ignore_index = True, axis = 0)
    data1 = data1[data1["HorsePower"].str.contains("0.0kWh") == False]
    data1 = data1[data1["PriceInText"].str.contains("Call") == False]
    data1.drop_duplicates(inplace=True)
    data1['Fuel'] = data1['Fuel'].replace('CNG', 'Petrol')
    data1.to_excel("Results.xlsx", index=False,sheet_name='Used Cars')
    data1.tail()
        
else:
    print("Results file does not exist. Creating one")
    df.to_excel("Results.xlsx", index=False,sheet_name='Used Cars')
    df.tail()

Link: https://www.pakwheels.com/used-cars/search/-/?page=1&q=xl 

Link: https://www.pakwheels.com/used-cars/search/-/?page=2&q=xl 

Link: https://www.pakwheels.com/used-cars/search/-/?page=3&q=xl 

No more Pages left


In [3]:
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")

        
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")


pw="admin"
connection = create_server_connection("localhost", "root", pw)
create_database_query="Create Database PakWheels"
create_database(connection,create_database_query)
create_table="""
CREATE TABLE PKWH (
    Name VARCHAR(100),
    PriceInText VARCHAR(20),
    Price INT,
    City VARCHAR(50),
    Year INT,
    Distance VARCHAR(20),
    Fuel VARCHAR(20),
    HorsePower VARCHAR(20),
    Transmission VARCHAR(20),
    Certified VARCHAR(20),
    ManagedBy VARCHAR(50)
);

"""
db="PakWheels"
connection = create_db_connection("localhost", "root", pw, db) # Connect to the Database
execute_query(connection, create_table) # Execute our defined query

MySQL Database connection successful
Error: '1007 (HY000): Can't create database 'pakwheels'; database exists'
MySQL Database connection successful
Error: '1050 (42S01): Table 'pkwh' already exists'


In [157]:
def insert_data_into_mysql(dataframe, table_name, connection):
    cursor = connection.cursor()
    try:
        for _, row in dataframe.iterrows():
            sql_query = f"SELECT COUNT(*) FROM {table_name} WHERE Name = %s AND PriceInText = %s AND Price = %s AND City = %s AND Year = %s AND Distance = %s AND Fuel = %s AND HorsePower = %s AND Transmission = %s  AND Certified = %s AND ManagedBy = %s"
            values = (row['Name'], row['PriceInText'], row['Price'], row['City'], row['Year'], row['Distance'], row['Fuel'], row['HorsePower'], row['Transmission'], row['Certified'], row['Managed By'])
            cursor.execute(sql_query, values)
            result = cursor.fetchone()[0]
            if result == 0:
                sql_query = f"INSERT INTO {table_name} (Name, PriceInText, Price, City, Year, Distance, Fuel, HorsePower, Transmission,Certified,ManagedBy) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                cursor.execute(sql_query, values)
        connection.commit()
        print("Data inserted successfully into MySQL table")
    except Error as e:
        print("Error inserting data into MySQL table", e)
    finally:
        cursor.close()

insert_data_into_mysql(data1, 'PKWH', connection)

Data inserted successfully into MySQL table


In [ ]:
# import logging

# # Set the log file path
# filename = 'test_log.log'  # Replace with the desired log file path on Windows

# # Logger
# print(filename)
# logger = logging.getLogger(__name__)
# logger.setLevel(logging.INFO)

# file_handler = logging.FileHandler(filename)
# file_handler.setLevel(logging.INFO)
# file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
# logger.addHandler(file_handler)

# def do_logging():
#     logger.info("test")


# if __name__ == '__main__':
#     do_logging()


In [ ]:
# def insert_data_into_mysql(dataframe, table_name, connection):
#     cursor = connection.cursor()
#     try:
#         for _, row in dataframe.iterrows():
#             sql_query = f"SELECT COUNT(*) FROM {table_name} WHERE Name = %s AND PriceInText = %s AND Price = %s AND City = %s AND Year = %s AND Distance = %s AND Fuel = %s AND HorsePower = %s AND Transmission = %s  AND Certified = %s AND ManagedBy = %s"
#             values = (row['Name'], row['Price'], row['Price2'], row['City'], row['Year'], row['Distance'], row['Fuel'], row['HorsePower'], row['Transmission'], row['Certified'], row['ManagedBy'])
#             cursor.execute(sql_query, values)
#             result = cursor.fetchone()[0]
#             if result == 0:
#                 sql_query = f"INSERT INTO {table_name} (Name, PriceInText, Price, City, Year, Distance, Fuel, HorsePower, Transmission,Certified,ManagedBy) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
#                 cursor.execute(sql_query, values)
#         connection.commit()
#         print("Data inserted successfully into MySQL table")
#     except Error as e:
#         print("Error inserting data into MySQL table", e)
#     finally:
#         cursor.close()

# insert_data_into_mysql(df2, 'PKWH', connection)

In [ ]:
# def delete_all_data_from_table(table_name, connection):
#     try:
#         cursor = connection.cursor()
#         delete_query = f"DELETE FROM {table_name}"
#         cursor.execute(delete_query)
#         connection.commit()
#         print("All data removed from the table.")
#     except Error as e:
#         print("Error deleting data from the table:", e)
#     finally:
#         cursor.close()
# delete_all_data_from_table('PKWH', connection)